2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1



<img align="left" src="../images/ThinBlueBar.png" /><br/>

# Lab 7b - Using Terraform Modules from github.com



<img align="left" src="../images/ThinBlueBar.png" width="400" /><br/>

## Background:

In this exercise, we will take a look at the example module:

https://github.com/mjbright/terraform-modules/tree/master/latest-ubuntu-ami

which allows to look up the latest "bionic" ubuntu ami.




<img align="left" src="../images/ThinBlueBar.png" width="400" /><br/>

## Tasks:
### 1. Make a directory called ‘lab7b’ underneath the labs directory.
### 2. Change into the directory.

### 3. Create a main.tf for the root-module which will call out to the '*latest-ubuntu-ami*' module

This module refers to "github.com/" as source and so will pull the code down from Github.

The module will be stored in our ~/dot.terraform directory - if our TF_DATA_DIR environment variable is set to that value.

Create a main.tf file there should containing:

In [5]:
cat main.tf


provider "aws" {
    region = var.region
}

variable "region" {
    description = "the AWS region to use"
}

variable ami_instance {
    default = "unset"
}

module "latest-ubuntu-ami" {
    # Reference module latest-ubuntu-ami in github repo github.com/mjbright/terraform-modules
    # NOTE: double-slash to reference subdirectory of the repo:  //modules/latest-ubuntu-ami
    source = "github.com/mjbright/terraform-modules//modules/latest-ubuntu-ami"
    
    # translates to:
    #   git::https://github.com/mjbright/terraform-modules.git
    # To verify try:
    #     rm -rf /home/student/dot.terraform/modules
    #     TF_LOG=trace terraform init |& grep fetch
    
    region = var.region
}

resource "aws_instance" "example" {
    ami = module.latest-ubuntu-ami.amis_latest_ubuntu_bionic_LTS

    instance_type = "t2.micro"
    vpc_security_group_ids = [aws_security_group.secgroup-user10.id]
}

resource "aws_security_group" "secgroup-user10" {
    name = "simple security group - user10"

#### terraform.tfvars

We will also create a terraform.tfvars file to provide a value for the region to use:

In [7]:
cat terraform.tfvars


region="us-west-1"



#### Root module

The above  is our root-module which makes a **call** to the 
    "*latest-ubuntu-ami*" module.

### 4. Initialize the root config

To use this module we must first download the module from github - this is done automatically for us when we call either ```terraform get``` or ```terraform init```

**Note:** ```terraform get``` differs in that it downloads modules, but not the Provider plugin.

#### terraform get

Let's perform a ```terraform get``` just to see this in action - it is not a ```necessary``` step as we will perform ```terraform init``` after anyway.

#### terraform init

In [11]:
terraform init

Initializing modules...
- latest-ubuntu-ami in /home/student/dot.terraform/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami

Initializing the backend...

Initializing provider plugins...
- Finding latest version of hashicorp/aws...
- Installing hashicorp/aws v4.49.0...
- Installed hashicorp/aws v4.49.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it

### 5. Study the module source files

Now that we have performed ```terraform init``` we see that the module was downloaded from github.com for us and was placed under the ```~/dot.terraform/modules/``` folder.

**Note:** the github repository is ```github.com/mjbright/terraform-modules/``` but we specified a path of ```github.com/mjbright/terraform-modules/latest-ubuntu-ami``` where our module source is located - there could be multiple module sources in that repository.

Look at the files under ```~/dot.terraform/modules/latest-ubuntu-ami/latest-ubuntu-ami```

In [12]:
find $TF_DATA_DIR/modules/latest-ubuntu-ami/modules

/home/student/dot.terraform/modules/latest-ubuntu-ami/modules
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/azure-instances
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/azure-instances/main.tf
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/azure-instances/tls_private_key.tf
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/azure-instances/outputs.tf
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/azure-instances/variables.tf
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/azure-instances/setup.sh
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami/main.tf
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami/vars.tf
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami/outputs.tf
/home/student/dot.terraform/modules/latest-ubuntu-ami/modules/d

In [13]:
cat $TF_DATA_DIR/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami/main.tf


data "aws_ami" "latest_ubuntu_bionic" {
  most_recent = true

  filter {
    name   = "name"
    values = ["ubuntu/images/hvm-ssd/ubuntu-bionic-18.04-amd64-server-*"]
  }

  filter {
    name   = "virtualization-type"
    values = ["hvm"]
  }

  owners = ["099720109477"] # Canonical
}




In [14]:
cat $TF_DATA_DIR/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami/vars.tf


variable "region" { }




In [15]:
cat $TF_DATA_DIR/modules/latest-ubuntu-ami/modules/latest-ubuntu-ami/outputs.tf


output  "amis_latest_ubuntu_bionic_LTS" { value = data.aws_ami.latest_ubuntu_bionic.id }



The output value *"amis_latest_ubuntu_bionic_LTS"* will obtain the latest bionic ubuntu image for your region, which was obtained from the "*aws_ami*" data source of the "*aws*" provider.


### 6.  Applying the 'root module' configuration

Apply the configuration and you should see at the end of the apply the ami-instance which was selected for your region.


In [18]:
terraform apply 

module.latest-ubuntu-ami.data.aws_ami.latest_ubuntu_bionic: Reading...
module.latest-ubuntu-ami.data.aws_ami.latest_ubuntu_bionic: Read complete after 1s [id=ami-05bdaab9cff831ca7]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_instance.example will be created
  + resource "aws_instance" "example" {
      + ami                                  = "ami-05bdaab9cff831ca7"
      + arn                                  = (known after apply)
      + associate_public_ip_address          = (known after apply)
      + availability_zone                    = (known after apply)
      + cpu_core_count                       = (known after apply)
      + cpu_threads_per_core                 = (known after apply)
      + disable_api_stop                     = (known after apply)
      + disable_api_termination              = (known after apply)

          + volume_type           = (known after apply)
        }
    }

  # aws_security_group.secgroup-user10 will be created
  + resource "aws_security_group" "secgroup-user10" {
      + arn                    = (known after apply)
      + description            = "Managed by Terraform"
      + egress                 = (known after apply)
      + id                     = (known after apply)
      + ingress                = [
          + {
              + cidr_blocks      = [
                  + "0.0.0.0/0",
                ]
              + description      = ""
              + from_port        = 22
              + ipv6_cidr_blocks = []
              + prefix_list_ids  = []
              + protocol         = "tcp"
              + security_groups  = []
              + self             = false
              + to_port          = 22
            },
        ]
      + name                   = "simple security group - user10"
      + name_prefix            = (known after apply)
      + owne

### 8. Cleanup

To destroy the formerly created AWS instance.

In [19]:
terraform destroy 

module.latest-ubuntu-ami.data.aws_ami.latest_ubuntu_bionic: Reading...
aws_security_group.secgroup-user10: Refreshing state... [id=sg-018669c5177c7e214]
module.latest-ubuntu-ami.data.aws_ami.latest_ubuntu_bionic: Read complete after 1s [id=ami-05bdaab9cff831ca7]
aws_instance.example: Refreshing state... [id=i-0b9cca4fc26840125]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # aws_instance.example will be destroyed
  - resource "aws_instance" "example" {
      - ami                                  = "ami-05bdaab9cff831ca7" -> null
      - arn                                  = "arn:aws:ec2:us-west-1:816376574968:instance/i-0b9cca4fc26840125" -> null
      - associate_public_ip_address          = true -> null
      - availability_zone                    = "us-west-1c" -> null
      - cpu_core_count                       = 1 -> null
  

                ]
              - description      = ""
              - from_port        = 22
              - ipv6_cidr_blocks = []
              - prefix_list_ids  = []
              - protocol         = "tcp"
              - security_groups  = []
              - self             = false
              - to_port          = 22
            },
        ] -> null
      - name                   = "simple security group - user10" -> null
      - owner_id               = "816376574968" -> null
      - revoke_rules_on_delete = false -> null
      - tags                   = {} -> null
      - tags_all               = {} -> null
      - vpc_id                 = "vpc-0c4ad4047839bc08f" -> null
    }

Plan: 0 to add, 0 to change, 2 to destroy.

Changes to Outputs:
  - ami_instance = "ami-05bdaab9cff831ca7" -> null
aws_instance.example: Destroying... [id=i-0b9cca4fc26840125]
aws_instance.example: Still destroying... [id=i-0b9cca4fc26840125, 10s elapsed]
aws_instance.example: Still destroying... [id=



<img align="left" src="../images/ThinBlueBar.png" /><br/>

# Summary

- In this section we saw another example of using modules

In this case we specfified a github.com repository as our module source.

The module uses a data_source to obtain a lastest ubuntu aws ami image according to our criteria

<!--



<img align="left" src="../images/ThinBlueBar.png" /><br/>

# **Stretch Goal:** Change region/ami

Run terraform apply with a different region, e.g. us-east-1, to verify that a different ami is proposed

```TF_VAR_region=us-east-1 terraform apply```
-->



<img align="left" src="../images/ThinBlueBar.png" /><br/>

# Stretch Goals



<img align="left" src="../images/ThinBlueBar.png" width="400" /><br/>

## Stretch Goal 1. Modify the module

... you're way ahead of the pack ... try this ...

Clone the module definition using ```git clone https://github.com/mjbright/terraform-modules/``` and place this under a local module directory, then modify your config to use this new module
- git clone the repo
- move the source to modules/mymodule
- modify the module definition to take extra input variables:
  - *release*  : to pull a specific Ubuntu release
  - *num_vms*  : to specify how many instances to create
  - *key_pair* : to pass a tls_private key to use

For *'release'*, you should be able to pass an argument like "focal-20.04", "bionic-18.04", "trusty-14.04" and return a candidate ami



<img align="left" src="../images/ThinBlueBar.png" width="400" /><br/>

## Stretch Goal 2. Create your own module

Create your own module to
- Create your own module to create multiple file instances
- experiment with passing different arguments to the module
- experiment with recuperating output values from the module
- add features such as
  - calculating file checksums
  - create a zip archive file
- Create your own github repository containing the module
- Use the module directly from github - as described at https://www.terraform.io/docs/language/modules/sources.html#github



<img align="left" src="../images/ThinBlueBar.png" width="400" /><br/>

## Stretch Goal 3. Investigate AWS Modules

- Investigate the AWS Modules here: https://registry.terraform.io/namespaces/terraform-aws-modules
- Create EC2 instances using: https://registry.terraform.io/modules/terraform-aws-modules/ec2-instance/aws/latest

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg) -->
<img src="../images/LOGO_v2_CROPPED.jpg" width="200" />